In [1]:
!pip install duckduckgo_search

You should consider upgrading via the 'F:\Project Shopify Langchain\lang\Scripts\python.exe -m pip install --upgrade pip' command.


# Regular Search

In [4]:
from duckduckgo_search import DDGS
import requests
import re
from bs4 import BeautifulSoup

In [5]:
ddg = DDGS()

In [8]:
def search(query, max_results = 3):
    results = ddg.text(query, max_results=max_results)
    return results

In [9]:
search("Top news headlines from india")

[{'title': 'India News | Latest India News Headlines & Live Updates from India ...',
  'href': 'https://timesofindia.indiatimes.com/india',
  'body': 'India News Today: Get latest India news and breaking news from India on Times of India. TOI brings the top news headlines from India on Politics, Elections, Current Affairs and more'},
 {'title': "India News | Today's latest updates and breaking news from India, Live ...",
  'href': 'https://www.ndtv.com/india',
  'body': "India News | Latest India News | Read latest and breaking news from India. Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood."},
 {'title': 'Latest News, Breaking News Today - Entertainment, Cricket, Business ...',
  'href': 'https://www.indiatoday.in/',
  'body': 'Check out the latest news from India and around the world. Latest India news on Bollywood, Politics, Business, Cricket, Technology and Travel.'}]

In [7]:
def search(query, max_results = 3):
    results = ddg.text(query, max_results=max_results)
    return [i['href'] for i in results]

# Scrape data

In [9]:
def scrape_info(url):
    """Scrape content fron given URL"""
    if not url:
        return "URL can not be found"
    
    #fetch data
    headers={"User-Agent": 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code!=200:
        return "Failed to retrieve the webpage"
    
    # parse result
    soup = BeautifulSoup(response.text,'html.parser')
    return soup

In [10]:
urls = search("Top news headlines from india")

In [11]:
scrape_info(urls[0])

<!DOCTYPE html>
<html class="no-js" lang="en"><head><link href="//static.toiimg.com" rel="preconnect"/><link href="//securepubads.g.doubleclick.net" rel="preconnect"/><link href="//pagead2.googlesyndication.com" rel="preconnect"/><link href="//ads.pubmatic.com" rel="preconnect"/><link crossorigin="anonymous" href="//assets.toiimg.com" rel="preconnect"/><link as="script" fetchpriority="high" href="https://assets.toiimg.com/js/til_prebid.js?v=14" rel="preload"/><link as="script" fetchpriority="high" href="https://c.amazon-adsystem.com/aax2/apstag.js" rel="preload"/><link as="script" fetchpriority="high" href="https://securepubads.g.doubleclick.net/tag/js/gpt.js?network-code=&lt;7176&gt;" rel="preload"/><link as="script" fetchpriority="high" href="https://timesofindia.indiatimes.com/toiads_prebid/minify-1.cms" rel="preload"/><link as="style" href="https://assets.toiimg.com/assets/11024.c6d19997.chunk.css" rel="preload" type="text/css"/><link as="style" href="https://assets.toiimg.com/asse

In [12]:
def scrape_info(url):
    """Scrape content fron given URL"""
    if not url:
        return "URL can not be found"
    
    #fetch data
    headers={"User-Agent": 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code!=200:
        return "Failed to retrieve the webpage"
    
    # parse result
    soup = BeautifulSoup(response.text,'html.parser')
    
    # Extract text
    data = []
    for tag in soup.find_all(['h1','h2','h3','p']):
        text = tag.get_text(" ",strip=True)
        data.append(text)

    data = "\n".join(data)
    data = re.sub(r'\s+',' ', data)
    return f"{url}\n {data}"

In [13]:
scrape_info(urls[0])

'https://timesofindia.indiatimes.com/india\n TOI India News Meghalaya CM Sangma Conrad and Deputy CM Prestone Tynsong met Union Home Minister Amit Shah in New Delhi to discuss the evolving situation in Bangladesh and its impact on the North East. With rising concerns among border residents, Sangma emphasized the commitment to border safety and maintaining peace. The Centre issued an advisory for private news channels to include date and timestamp on visuals of disasters and accidents. The Ministry stressed that continuous replay of old footage can mislead viewers about the current situation, causing confusion and panic. This follows extensive coverage of recent landslides in Wayanad and Himachal Pradesh. Trending Topics Videos Popular India Stories More India News Quick Links'

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import time

def search_and_scrape(query, max_results=3):
    """Perform a search and scrape content from the resulting URLs."""
    
    start_time = time.time()
    
    # Perform the search and retrieve URLs
    results = ddg.text(query, max_results=max_results)
    
    # Prepare the structured output
    output = {
        'query': query,
        'follow_up_questions': None,
        'answer': None,
        'images': [],
        'results': [],
        'response_time': None
    }
    
    # Scrape content from each result
    for result in results:
        url = result['href']
        title = result.get('title', 'No title')
        
        # Scrape the webpage
        headers = {"User-Agent": 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            data = []
            
            for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
                text = tag.get_text(" ", strip=True)
                data.append(text)
            
            content = " ".join(data)
            content = re.sub(r'\s+', ' ', content)
            score = result.get('score', 0)  # Assuming 'score' is part of the search result
            
            output['results'].append({
                'title': title,
                'url': url,
                'content': content,
                'score': score,
                'raw_content': None
            })
        else:
            output['results'].append({
                'title': title,
                'url': url,
                'content': 'Failed to retrieve the webpage',
                'score': 0,
                'raw_content': None
            })
    
    # Calculate the response time
    output['response_time'] = round(time.time() - start_time, 2)
    
    return output


In [7]:
response = search_and_scrape(query="recent developments in bangladesh")
print(response)

{'query': 'recent developments in bangladesh', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Bangladesh Overview: Development news, research, data | World Bank', 'url': 'https://www.worldbank.org/en/country/bangladesh/overview', 'content': "With 189 member countries, staff from more than 170 countries, and offices in over 130 locations, the World Bank Group is a unique global partnership: five institutions working for sustainable solutions that reduce poverty and build shared prosperity in developing countries. The World Bank Group works in every major area of development. We provide a wide array of financial products and technical assistance, and we help countries share and apply innovative knowledge and solutions to the challenges they face. We face big challenges to help the world’s poorest people and ensure that everyone sees benefits from economic growth. Data and research help us understand these challenges and set priorities, share knowledge o